In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 25 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

11 April 2023 ; 13:31:54


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,1,1
12,MarJon Beauchamp,SF,22,MIL,52,11,13.5,1.8,4.7,0.395,...,3.63,116.13,112.64,3.49,3.61,116.12,112.64,3.48,1,1
18,A.J. Lawson,SG,22,DAL,15,0,7.2,1.5,2.9,0.500,...,0.07,117.63,117.56,0.06,-0.14,117.54,117.70,-0.16,1,1
26,Jaden Hardy,SG,20,DAL,48,5,14.8,3.0,6.9,0.438,...,0.07,117.63,117.56,0.06,-0.14,117.54,117.70,-0.16,1,1
39,AJ Griffin,SF,19,ATL,72,12,19.5,3.4,7.4,0.465,...,0.29,117.45,117.16,0.29,0.32,117.66,117.34,0.32,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.8,ORL,14.9,0.341667,2.733333,6.7,20.0,13,0.253651
1,Walker Kessler,23.0,UTA,21.5,2.327027,7.867568,4.0,9.2,12,0.190390
2,Bennedict Mathurin,28.5,IND,13.1,-1.051282,1.892308,5.3,16.7,11,0.108355
3,Jalen Duren,24.9,DET,17.3,0.611940,5.507463,3.9,9.1,15,0.103650
4,Jaden Ivey,31.1,DET,12.1,-0.775676,0.000000,5.5,16.3,15,0.092891


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)